In [ ]:
# %load ./init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import os
import sys
import logging
import warnings
import numpy as np
import astropy as ap
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import h5py
import tqdm.notebook as tqdm

import kalepy as kale
import kalepy.utils
import kalepy.plot

import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})

log = holo.log
log.setLevel(logging.INFO)

# Quick Start

In [ ]:
# ---- Construct S.A.M. with default settings ----
sam = holo.sam.Semi_Analytic_Model()

In [ ]:
# ---- Choose frequencies, and calculate GWB ----
fobs = utils.nyquist_freqs()
gwb = sam.gwb(fobs, realize=10)    # calculate 10 different realizations

In [ ]:
# ---- Plot GWB ----

fig, ax = plt.subplots(figsize=[15, 8])
ax.set(xscale='log', xlabel='Frequency [1/yr]', yscale='log', ylabel='Characteristic Strain')
ax.grid(alpha=0.2)

yy = 2e-16 * np.power(fobs, -2.0/3.0)
ax.plot(fobs, yy, 'k--', alpha=0.25, lw=2.0)
ax.plot(fobs, np.median(gwb, axis=-1), 'k-')
for pp in [50, 98]:
    percs = pp / 2
    percs = [50 - percs, 50 + percs]
    ax.fill_between(fobs, *np.percentile(gwb, percs, axis=-1), alpha=0.25, color='b')

plt.show()

# build SAM component-by-component

In [ ]:
gsmf = holo.sam.GSMF_Schechter()
gpf = holo.sam.GPF_Power_Law()
gmt = holo.sam.GMT_Power_Law()
mmbulge = holo.sam.MMBulge_Simple()

sam = holo.sam.Semi_Analytic_Model(gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge)

In [ ]:
# ---- Calculate GWB Amplitude distribution at 1/yr ----
ayr = sam.gwb(1.0, realize=100)

In [ ]:
fig, ax = plt.subplots(figsize=[10, 6])
ax.set(xlabel=r'$\log_{10}(A_\mathrm{yr})$', ylabel='Probability Density')
ax.grid(alpha=0.2)
kale.dist1d(np.log10(ayr), density=True, confidence=True)

plt.show()

## Plot GWB Amplitude Distribution vs. M-MBulge parameters

In [ ]:
alpha_list = [0.75, 1.0, 1.25]
norm_list = np.logspace(7, 9, 11)
NREALS = 300
FREQ = 1.0   # [1/yr]

dist_mmb = np.zeros((len(alpha_list), norm_list.size, NREALS))

for aa, alpha in enumerate(tqdm.tqdm(alpha_list)):
    for nn, norm in enumerate(tqdm.tqdm(norm_list, leave=False)):
        mmbulge = holo.sam.MMBulge_Simple(mass_norm=norm, malpha=alpha)
        sam = holo.sam.Semi_Analytic_Model(gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge)
        dist_mmb[aa, nn, :] = sam.gwb(FREQ, realize=NREALS)

In [ ]:
fig, ax = plt.subplots(figsize=[6, 4])
ax.set(xscale='log', xlabel='M-MBulge Mass', yscale='log', ylabel=r'GWB $A_\mathrm{yr}$')
ax.grid(alpha=0.2)

dist = dist_mmb

for aa, dd in zip(alpha_list, dist):
    med = np.median(dd, axis=-1)
    cc, = ax.plot(norm_list, med, label=aa)
    cc = cc.get_color()
    ax.fill_between(norm_list, *np.percentile(dd, [25, 75], axis=-1), color=cc, alpha=0.25)
    
plt.legend(title='slope')
plt.show()

## Plot GWB Amplitude vs. Schecter Mass Parameter

In [ ]:
mz = [0.0, 1e10, 1e11]
m0 = np.logspace(10, 12, 7)
NREALS = 100
FREQ = 1.0   # [1/yr]

dist = np.zeros((len(mz), len(m0), NREALS))

for aa, _mz in enumerate(tqdm.tqdm(mz)):
    for nn, _m0 in enumerate(tqdm.tqdm(m0, leave=False)):
        gsmf = holo.sam.GSMF_Schechter(mref0=_m0, mrefz=_mz)
        sam = holo.sam.Semi_Analytic_Model(gsmf=gsmf, gpf=gpf, gmt=gmt, mmbulge=mmbulge)
        dist[aa, nn, :] = sam.gwb(FREQ, realize=NREALS)

In [ ]:
fig, ax = plt.subplots(figsize=[16, 6])
ax.set(xscale='log', xlabel='Schechter Mass', yscale='log', ylabel=r'GWB $A_\mathrm{yr}$')
ax.grid(alpha=0.2)

for aa, dd in zip(mz, dist):
    med = np.median(dd, axis=-1)
    lab = f"$10^{{{np.log10(aa):+.1f}}}$" if aa > 0.0 else f"${aa:+.1f}$"
    cc, = ax.plot(m0, med, label=lab)
    cc = cc.get_color()
    ax.fill_between(m0, *np.percentile(dd, [25, 75], axis=-1), color=cc, alpha=0.25)
    
plt.legend(title='$M_0(z)$')
plt.show()

# Discretize Population

In [ ]:
sam = holo.sam.Semi_Analytic_Model()

In [ ]:
fobs = utils.nyquist_freqs(20.0, 0.1)

edges, number, strain = sam.number_at_gw_fobs(fobs, limit_merger_time=2.0)
log_edges_fobs = [np.log10(edges[0]), edges[1], edges[2], np.log10(edges[3])]

num = number.sum()
print(f"{num:.4e}")

## Sample using kalepy

In [ ]:
vals, weights = holo.sam.sample_sam(
    sam, fobs, sample_threshold=10.0, cut_below_mass=None, limit_merger_time=2.0
)

In [ ]:
bins = (sam.mtot, sam.mrat)
nums = number[..., 0].sum(axis=-1)
nums[nums < 1.0] = 0.0

norm = mpl.colors.LogNorm(nums[nums > 0].min()/2, 2*nums.max())

fig, axes = plt.subplots(figsize=[15, 6], ncols=2)

for ax in axes:
    ax.set(xscale='log', xlabel='log10 Mass Primary', ylabel='Mass Ratio')

ax = axes[0]
ax.set(title='Continuous')
pcm = ax.pcolormesh(*bins, nums.T, norm=norm)
plt.colorbar(pcm, ax=ax, orientation='horizontal')


ax = axes[1]
ax.set(title='Discrete (sampled)')
idx = (vals[3] < fobs[1])
hist, *_ = np.histogram2d(vals[0, idx], vals[1, idx], bins=bins, weights=weights[idx])

pcm = ax.pcolormesh(*bins, hist.T, norm=norm)
plt.colorbar(pcm, ax=ax, orientation='horizontal')

plt.show()


## Calculate GWs

In [ ]:
gwf_freqs, gwf, gwb = holo.sam._gws_from_samples(vals, weights, fobs)

In [ ]:
fig, ax = plt.subplots()
ax.set(xscale='log', yscale='log')
ax.grid(alpha=0.25)

idx = (gwf > gwb)
ax.scatter(gwf_freqs[idx], gwf[idx], color='r', s=20, alpha=0.5)
ax.scatter(gwf_freqs[~idx], gwf[~idx], edgecolor='r', facecolor='none', s=20, alpha=0.5)

xx = kale.utils.midpoints(fobs)
ax.plot(xx, gwb, 'k-')

amp = 10e-16
yy = amp * np.power(xx, -2/3)
ax.plot(xx, yy, 'k--', alpha=0.25)

plt.show()